In [13]:
!pip install google-cloud-documentai

     |████████████████████████████████| 105 kB 27.1 MB/s eta 0:00:01


In [17]:
from google.cloud import documentai_v1beta2 as documentai

In [18]:
client = documentai.DocumentUnderstandingServiceClient()

In [21]:
input_uri = "gs://jayadeep-etl-platform/docuai/demo-doc.pdf"
gcs_source = documentai.types.GcsSource(uri=input_uri)

In [22]:
input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type="application/pdf"
    )

In [25]:
table_extraction_params = documentai.types.TableExtractionParams(
        enabled=True)
project_id = "jayadeep-etl-platform"
parent = "projects/{}/locations/us".format(project_id)
request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        table_extraction_params=table_extraction_params,
    )

document = client.process_document(request=request)

In [ ]:
def _get_text(el):
        """Convert text offset indexes into text snippets."""
        response = ""
        # If a text segment spans several lines, it will
        # be stored in different text segments.
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response.replace('\n','')

In [40]:
for page in document.pages:
        print("Page number: {}".format(page.page_number))
        for table_num, table in enumerate(page.tables):
            print("Table {}: ".format(table_num))
            for row_num, row in enumerate(table.header_rows):
                cells = "\t".join([_get_text(cell.layout) for cell in row.cells])
                print("Header Row {}: {}".format(row_num, cells))
            for row_num, row in enumerate(table.body_rows):
                #print (row)
                cells = "\t".join([_get_text(cell.layout) for cell in row.cells])
                #print (cells)
                print("Row {}: {}".format(row_num, cells))

Page number: 1
Table 0: 
Header Row 0: Strength: This text needs to be extracted	
Row 0: Name: ABC	Quantity: 250
Row 1: Issue No: PQ-43-97A	Date: 01-01-2020
Table 1: 
Header Row 0: Sequence	Frames	Tracking withoutdistance cue P/R/F 	Tracking withdistance cue P/R/F
Row 0: UAV1	616	0.90/0.90/0.90	0.94/0.94/0.94
Row 1: UAV 2	1833	0.92/0.88/0.90	0.96/0.99/0.97
Row 2: Traffic 1	156	0.88/0.91/0.89	0.96/0.98/0.97
Row 3: Traffic 2	227	0.85/0.90/0.87	0.97/0.97/0.97
Row 4: Pedestrian 2	338	0.93/0.95/0.94	1.00/1.00/1.00
